VADER Sentiment Analysis
1. http://www.nltk.org/howto/sentiment.html
2. https://github.com/cjhutto/vaderSentiment

VADER Sentiment Analysis. VADER (Valence Aware Dictionary and sEntiment Reasoner) is a lexicon and rule-based sentiment analysis tool that is specifically attuned to sentiments expressed in social media, and works well on texts from other domains.

Hutto, C.J. & Gilbert, E.E. (2014). VADER: A Parsimonious Rule-based Model for Sentiment Analysis of Social Media Text. Eighth International Conference on Weblogs and Social Media (ICWSM-14). Ann Arbor, MI, June 2014.

In [56]:
from nltk.sentiment import SentimentAnalyzer
from nltk.sentiment.util import *
nltk.download('vader_lexicon')

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /Users/cesar/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


True

In [55]:
# Open file and read only tweets extracted
hashtag = 'Macron'

tweets = []
with open(hashtag+'.json', 'r') as f:
    for line in f:
        tweet = json.loads(line)
        tokens = tweet['text']
        tweets.append(tokens)

'RT @simonjhix: #Macron v #LePen =&gt; end of the economic left-right of C20th, &amp; a return to the left-right of C19th, of liberalism v conserva…'

In [57]:
sid = SentimentIntensityAnalyzer()

In [58]:
for sentence in tweets:
    print(sentence)
    ss = sid.polarity_scores(sentence)
    for k in sorted(ss):
        print('{0}: {1}, '.format(k, ss[k]), end='')
        print()

RT @simonjhix: #Macron v #LePen =&gt; end of the economic left-right of C20th, &amp; a return to the left-right of C19th, of liberalism v conserva…
compound: 0.0, 
neg: 0.0, 
neu: 1.0, 
pos: 0.0, 
RT @nicolasbayfn: "Le clivage a le mérite d'être clair.@MLP_officiel défend la nation, Emmanuel #Macron défend la disparition de toute prot…
compound: 0.0, 
neg: 0.0, 
neu: 1.0, 
pos: 0.0, 
RT @colvieux: I discorsi di #Macron li scrive #Renzi?  #maratonamentana
compound: 0.0, 
neg: 0.0, 
neu: 1.0, 
pos: 0.0, 
RT @carolinedehaas: Et là, devant sa télé, la France se demande ce qu'elle a fait. #Macron #Vide #Mauvais
compound: 0.0, 
neg: 0.0, 
neu: 1.0, 
pos: 0.0, 
EXACTEMENT MERCI ERIC T UN BG JTM https://t.co/FY3cu9JADZ
compound: 0.0, 
neg: 0.0, 
neu: 1.0, 
pos: 0.0, 
RT @JWGOP: Vital we turn the tide internationally on this dangerous white nationalist movement, making the Trump election truly the outlier…
compound: 0.1217, 
neg: 0.136, 
neu: 0.674, 
pos: 0.191, 
#Estrosi il s'est fait piraté s